In [54]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob
import ast
from skimage import morphology
import math

## Labels

In [55]:
#Get base labels
base_path_labels = "../data/raw/Test/6.0_Glaucoma_Decision"

df_bosch = pd.read_csv(f'{base_path_labels}/Glaucoma_Decision_Comparison_Bosch_majority.csv')
#Rename label columns to fit the others
df_bosch.rename({"Majority Decision":"Glaucoma Decision"},axis=1,inplace=True)

df_forus = pd.read_csv(f'{base_path_labels}/Glaucoma_Decision_Comparison_Forus_majority.csv')
df_remidio = pd.read_csv(f'{base_path_labels}/Glaucoma_Decision_Comparison_Remidio_majority.csv')

#Create ImageID as the real file name and extension
df_bosch["ImageID"] = df_bosch["Images"].apply(lambda x:x.split("-")[0].replace("jpg","JPG"))
df_forus["ImageID"] = df_forus["Images"].apply(lambda x:x.split("-")[0].replace("jpg","png"))
df_remidio["ImageID"] = df_remidio["Images"].apply(lambda x:x.split("-")[0].replace("tif","JPG"))


#Add device before concat all labels
df_bosch["Camera"] = "Bosch"
df_forus["Camera"] = "Forus"
df_remidio["Camera"] = "Remidio"
 
#Concat all labels together
df_all_source = pd.concat([df_bosch,df_forus,df_remidio])


#Remove labels for images not present in the data
base_path_images = "../data/raw/Test/1.0_Original_Fundus_Images"
images_name = [p.split("/")[-1] for p in glob.glob(f"{base_path_images}/*/*")]
missing_images = ~(df_all_source["ImageID"].isin(images_name))
df_all_source_present_images = df_all_source[~missing_images]

#Change the label to binary value instead of "NORMAL" and "GLAUCOMA SUSPECT"
df_all_source_present_images["Onehot"] = df_all_source_present_images["Glaucoma Decision"].apply(lambda x: [1 if x == "GLAUCOMA SUSPECT" else 0])

#Add PatientID to avoid images from a same patient in different split, only useful for Bosch images starting with PXX_ where XX could be the PatientID
df_all_source_present_images["PatientID"] = df_all_source_present_images["ImageID"].apply(lambda x: x.split("_")[0] if x.startswith("P") else x[:-4])

#Only keep necessary columns
df_processed = df_all_source_present_images[["ImageID","PatientID","Camera","Onehot"]]

/tmp/ipykernel_150412/2270055706.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all_source_present_images["Onehot"] = df_all_source_present_images["Glaucoma Decision"].apply(lambda x: [1 if x == "GLAUCOMA SUSPECT" else 0])
/tmp/ipykernel_150412/2270055706.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all_source_present_images["PatientID"] = df_all_source_present_images["ImageID"].apply(lambda x: x.split("_")[0] if x.startswith("P") else x[:-4])


In [56]:
df_processed

,ImageID,PatientID,Camera,Onehot
86,P10_Image2.JPG,P10,Bosch,[0]
87,P11_Image1.JPG,P11,Bosch,[0]
88,P11_Image2.JPG,P11,Bosch,[0]
89,P12_Image1.JPG,P12,Bosch,[0]
90,P12_Image2.JPG,P12,Bosch,[0]
...,...,...,...,...
259,IMG_4547.JPG,IMG_4547,Remidio,[1]
260,IMG_4558.JPG,IMG_4558,Remidio,[0]
261,IMG_4559.JPG,IMG_4559,Remidio,[0]
262,IMG_4560.JPG,IMG_4560,Remidio,[0]
